In [1]:
import pickle
import json
import pandas as pd
import numpy as np
import xgboost

In [2]:
nsl_v2 = xgboost.XGBClassifier()

In [6]:
nsl_v2.load_model('../models/nsql_model_v2.pkl')

XGBoostError: std::bad_alloc

In [8]:
nsl_v2 = pickle.load(open('../models/nsql_model_v2.pkl', "rb"))


[19:09:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/learner.cc:1040: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[19:09:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/learner.cc:749: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[19:09:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/learner.cc:438: 
  If you are loading a serialized model (like pickle in Pytho

In [11]:
config = nsl_v2.save_config()

AttributeError: 'XGBClassifier' object has no attribute 'save_config'

In [ ]:
input_keys = json.load(open('../data/nsql_sample_input.json', 'rb'))

In [10]:
input_keys.keys()

dict_keys(['application_id', 'estimated_monthly_revenue', 'incoming_ach_payments', 'outgoing_ach_and_checks', 'check_deposit_amount', 'outgoing_wire_transfers', 'incoming_wire_transfer', 'business_type', 'email_domain', 'current_bank', 'industry_category_name', 'iovation_device_type', 'iovation_device_timezone', 'carrier', 'socure_sigma', 'socure_phonerisk', 'socure_emailrisk', 'socure_reason_code', 'socure_phonerisk_reason_code', 'socure_emailrisk_reason_code', 'screen_width_mean', 'screen_height_mean'])